# Model Accuracy - 99.9%

- ### CNN Model using Keras tensorflow Backend.

In [ ]:
import csv
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau

print("Setup Complete!")

In [ ]:
def get_data(filename):
    with open(filename) as training_file:
        file = csv.reader(training_file, delimiter = ",")
        images = []
        labels = []
        ignore = 1
        for row in file:
            if ignore == 1:
                ignore = 0
                continue
            labels.append(row[0])
            images.append(np.array_split(row[1:],28))
    return np.array(images).astype("int32"), np.array(labels).astype("int32")


In [ ]:
train_path = '/kaggle/input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv'
test_path = '/kaggle/input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv'

x_train, y_train = get_data(train_path)
x_test, y_test = get_data(test_path)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

In [ ]:
train_data_gen = ImageDataGenerator(rescale = 1./255,
      rotation_range=30,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest'
    )

validation_data_gen = ImageDataGenerator(rescale = 1./255)

print(x_train.shape)
print(x_test.shape)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu,input_shape=(28, 28, 1)),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu,padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation=tf.nn.relu,padding = 'Same'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(256,activation=tf.nn.relu),
    tf.keras.layers.Dense(25, activation = tf.nn.softmax)
])

model.summary()

In [ ]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

training_generator = train_data_gen.flow(x_train, y_train_cat, batch_size=64)
validation_generator = validation_data_gen.flow(x_test, y_test_cat, batch_size=64)

model.compile(loss = 'categorical_crossentropy', optimizer= tf.keras.optimizers.Adam(), metrics=['acc'])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=2,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.000003)

In [ ]:
history = model.fit(training_generator,
                    epochs = 40,
                    verbose = 1,
                   validation_data = validation_generator,
                   callbacks=[learning_rate_reduction])

In [ ]:
%matplotlib inline
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
%matplotlib inline
acc = history.history['acc'][13:]
val_acc = history.history['val_acc'][13:]
loss = history.history['loss'][13:]
val_loss = history.history['val_loss'][13:]

epochs = range(13,13+len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
get_ipython().run_cell_magic('javascript', '', '<!-- Save the notebook -->\nIPython.notebook.save_checkpoint();')